# Standardizing the data

I've already formatted the data into parquet files (my computer is old and excel is slow).  See scripts/format_data.ipynb

This notebook is taking that raw data and standardizing it.  Once completed it will be exported to data/processed/

Another thing this notebook does is look further into uniquness in the rows.  A basic data quality report was created on the raw excel data (see scripts/case_study_dqr.ipynb), and this notebook answers some questions raised from it.

Overall the data quality is good (Thanks to Axuall), all I'm really doing is applying stndz naming conventions to table/column names and adding a unique key to the claims counts table in case I need one later on.  But here's a complete list of this notebook:

## 1: Renaming Stuff for Simplicity
- Just applying snake_case to make it easier to find what I need later

## 2: Checking for Unique Bits
- We checked to see if some columns had special, one-of-a-kind info in each row

## 3: Counting and Sorting
- Just taking stock to get an idea of what's more common and what's rare and making sure dates make sense, etc.

## 4: Gave the claims table a primary key
- Created a 'super key' for claims counts by mashing together important bits from several columns

## 5: Packing Up and Export the Data
- Export back into Parquet files and stored them in data/processed

And that's pretty much it!


In [1]:
import os
import pandas as pd

def load_parquet_files(directory):
    """
    Loads all Parquet files from a specified directory into a dictionary of DataFrames.
    """
    dataframes = {}
    if not os.path.exists(directory):
        print(f"Directory not found: {directory}")
        return dataframes

    for filename in os.listdir(directory):
        if filename.endswith('.parquet'):
            file_path = os.path.join(directory, filename)
            try:
                df = pd.read_parquet(file_path)
                dataframes[os.path.splitext(filename)[0]] = df
                print(f"Loaded {filename}")
            except Exception as e:
                print(f"Failed to load {filename}: {e}")
    
    return dataframes

# Transformed excel into parquet so I could work with it better on my old machine
parquet_directory = os.path.join(os.getcwd(), '..', 'data', 'interim')

# Load Parquet files into dataframes dictionary
dataframes = load_parquet_files(parquet_directory)

# Check the loaded DataFrames
for name, df in dataframes.items():
    print(f"DataFrame loaded from {name}: {df.shape[0]} rows, {df.shape[1]} columns")


Loaded Provider-Facility Affiliated Cl_data.parquet
Loaded Facilities_data.parquet
Loaded Provider Demographics_data.parquet
DataFrame loaded from Provider-Facility Affiliated Cl_data: 108280 rows, 8 columns
DataFrame loaded from Facilities_data: 11 rows, 39 columns
DataFrame loaded from Provider Demographics_data: 8459 rows, 12 columns


In [2]:
# Check some of the data manually
for name, df in dataframes.items():
    print(f"Info for DataFrame: {name}")
    print(df.info())
    print(f"\nFirst few rows of DataFrame: {name}")
    print(df.head())

    # separator
    print("\n" + "="*50 + "\n")  


Info for DataFrame: Provider-Facility Affiliated Cl_data
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108280 entries, 0 to 108279
Data columns (total 8 columns):
 #   Column                        Non-Null Count   Dtype 
---  ------                        --------------   ----- 
 0   Clinician NPI                 108280 non-null  int64 
 1   Facility NPI                  108280 non-null  int64 
 2   Healthcare Organization Name  108280 non-null  string
 3   class                         108280 non-null  string
 4   year                          108280 non-null  int64 
 5   quarter                       108280 non-null  int64 
 6   total_claims                  108280 non-null  int64 
 7   total_patients                108280 non-null  int64 
dtypes: int64(6), string(2)
memory usage: 6.6 MB
None

First few rows of DataFrame: Provider-Facility Affiliated Cl_data
   Clinician NPI  Facility NPI                   Healthcare Organization Name  \
0     1225617533    1679525919           

In [3]:
# standardize df names
name_mapping = {
    'Provider-Facility Affiliated Cl_data': 'claims',
    'Facilities_data': 'facilities',
    'Provider Demographics_data': 'demographics'
}

# rename
renamed_dataframes = {name_mapping.get(name, name): df for name, df in dataframes.items()}


In [9]:
# standardize column names
import re

def to_snake_case(name):
    """
    Convert a string to snake_case.
    """
    s1 = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', name.replace(" ", ""))
    return re.sub('([a-z0-9])([A-Z])', r'\1_\2', s1).lower()
    
for df_name, df in renamed_dataframes.items():
    # Standardize each column name to snake_case
    df.columns = [to_snake_case(col) for col in df.columns]


In [10]:
# Check for unique values in each column
for name, df in renamed_dataframes.items():
    print(f"Checking unique values in DataFrame: {name}")
    total_rows = len(df)

    for column in df.columns:
        if df[column].nunique() == total_rows:
            print(f"Column '{column}' has unique values for each row.")
    
    # separator
    print("\n" + "="*50 + "\n")  


Checking unique values in DataFrame: claims


Checking unique values in DataFrame: facilities
Column 'enrollment_id' has unique values for each row.
Column 'npi' has unique values for each row.
Column 'ccn' has unique values for each row.


Checking unique values in DataFrame: demographics
Column 'npi' has unique values for each row.




In [11]:
# Claims has no columns with unique values
#  makes sense since it's counts by year, quarter

renamed_dataframes['claims'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108280 entries, 0 to 108279
Data columns (total 8 columns):
 #   Column                        Non-Null Count   Dtype 
---  ------                        --------------   ----- 
 0   clinician_npi                 108280 non-null  int64 
 1   facility_npi                  108280 non-null  int64 
 2   healthcare_organization_name  108280 non-null  string
 3   class                         108280 non-null  string
 4   year                          108280 non-null  int64 
 5   quarter                       108280 non-null  int64 
 6   total_claims                  108280 non-null  int64 
 7   total_patients                108280 non-null  int64 
dtypes: int64(6), string(2)
memory usage: 6.6 MB


In [12]:
# Function to check for multiple entries in grouped data
def check_multiple_entries(df, grouping_columns):
    grouped = df.groupby(grouping_columns).size()
    groups_with_multiple_entries = grouped[grouped > 1]

    if groups_with_multiple_entries.empty:
        print(f"{', '.join(grouping_columns)} has unique entries.")
    else:
        print(f"Groups with multiple entries found for {', '.join(grouping_columns)}:")
        print(groups_with_multiple_entries)

claims_df = renamed_dataframes['claims']

# Check for clinician_npi, year, and quarter
check_multiple_entries(claims_df, ['clinician_npi', 'year', 'quarter'])

# Check for facility_npi, year, and quarter
check_multiple_entries(claims_df, ['facility_npi', 'year', 'quarter'])

# Check for clinician_npi, facility_npi, year, and quarter
check_multiple_entries(claims_df, ['clinician_npi', 'facility_npi', 'year', 'quarter'])



Groups with multiple entries found for clinician_npi, year, quarter:
clinician_npi  year  quarter
1003018763     2022  3          2
1003031188     2022  4          2
1003051715     2021  1          2
                     2          3
                     3          2
                               ..
1992954390     2022  4          2
1992974968     2021  2          2
               2022  1          2
                     3          2
                     4          2
Length: 19084, dtype: int64
Groups with multiple entries found for facility_npi, year, quarter:
facility_npi  year  quarter
1003854951    2021  1            9
                    2            8
                    3            9
                    4            9
              2022  1            9
                              ... 
1710951801    2021  4          191
              2022  1          190
                    2          205
                    3          176
                    4          128
Length: 88, dtype: 

In [13]:
# I'll be focusing on year, qtr, counts...
for column in claims_df.columns:
    print(f"Value counts in column '{column}':")
    print(claims_df[column].value_counts())
    print("\n" + "="*50 + "\n")  


Value counts in column 'clinician_npi':
clinician_npi
1811034622    33
1619943388    32
1962622878    29
1548240286    29
1699089631    29
              ..
1861883795     1
1255921656     1
1023621216     1
1992437925     1
1700811932     1
Name: count, Length: 16839, dtype: int64


Value counts in column 'facility_npi':
facility_npi
1679525919    52940
1043397292    22741
1215989611    13155
1700828852     7435
1194763045     5241
1518042175     2628
1033161948     2352
1710951801     1550
1063871820      129
1003854951       78
1467489682       31
Name: count, dtype: int64


Value counts in column 'healthcare_organization_name':
healthcare_organization_name
THE CLEVELAND CLINIC FOUNDATION                            52940
UNIVERSITY HOSPITALS CLEVELAND MEDICAL CENTER              25369
FAIRVIEW HOSPITAL                                          13155
THE METROHEALTH SYSTEM                                      7466
LUTHERAN HOSPITAL                                           5319
CLEVELA

In [16]:
def create_primary_key(df, columns, new_key_name):
    """
    Create a concatenated primary key from specified columns
    """
    # Check if columns exist
    missing_columns = [col for col in columns if col not in df.columns]
    if missing_columns:
        raise ValueError(f"Missing columns in DataFrame: {missing_columns}")

    # Convert columns to strings
    df[new_key_name] = df[columns].astype(str).apply(lambda row: '-'.join(row), axis=1)

    return df

# Column names
columns_to_concatenate = ['clinician_npi', 'facility_npi', 'year', 'quarter']

# Create primary key for claims counts
try:
    renamed_dataframes['claims'] = create_primary_key(renamed_dataframes['claims'], columns_to_concatenate, 'claim_counts_id')
    print("Primary key created successfully.")
except ValueError as e:
    print(f"Error: {e}")


Primary key created successfully.


In [17]:
def export_to_parquet_files(dataframes, directory):
    """
    Exports each DataFrame in a dictionary to a Parquet file in the processed data directory.
    """
    if not os.path.exists(directory):
        os.makedirs(directory)

    for name, df in dataframes.items():
        file_path = os.path.join(directory, f"{name}.parquet")
        try:
            df.to_parquet(file_path)
            print(f"Exported {name} to {file_path}")
        except Exception as e:
            print(f"Failed to export {name}: {e}")

# Export standardized data
export_directory = os.path.join(os.getcwd(), '..', 'data', 'processed')
export_to_parquet_files(renamed_dataframes, export_directory)


Exported claims to /Users/bbubnick/projects/claims_case_study/scripts/../data/processed/claims.parquet
Exported facilities to /Users/bbubnick/projects/claims_case_study/scripts/../data/processed/facilities.parquet
Exported demographics to /Users/bbubnick/projects/claims_case_study/scripts/../data/processed/demographics.parquet
